In [2]:
%load_ext autoreload
%autoreload

import sys
sys.path.insert(0, '..')

In [3]:
import pandas
import numpy

In [4]:

def load_microphones():
    path = '../references/microphones/mic_specifications.csv'
    df = pandas.read_csv(path, header=1)
    df = df.set_index('part')
    df['noisefloor'] = df.sensitivity - df.snr
    df['noise_spl'] = 94 - df.snr
    return df

mics = load_microphones()
mics

,cost,current,sensitivity,impedance,snr,thd,highpass,comments,noisefloor,noise_spl
part,,,,,,,,,,
LMA2718T421-OA5,0.0578,0.20,-42,300,55,0.10,200,NaN,-97,39
LMA2718T381-OC1,0.1468,0.20,-38,300,62,0.10,50,Similar to OA5. But has specified tolerances o...,-100,32
LMA2718T381-OY1,0.2753,0.20,-38,300,64,0.15,20,Similar to OC1. But has specified tolerances o...,-102,30
S15OT421-005,0.0888,0.15,-42,400,59,1.00,50,THD is max. Freq response only specified down ...,-101,35


In [5]:

def print_mics(mics):
    df = mics.copy()
    df['current_ua'] = df.current * 1000
    columns = {
        'current_ua': 'Current (uA)',
        'noise_spl': 'Self noise (dBa SPLa)',
        #'sensitivity': 
    }
    df = df.rename(columns=columns)
    df = df.sort_values('cost')
    return df[columns.values()].to_html()

print_mics(mics)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Current (uA)</th>\n      <th>Self noise (dBa SPLa)</th>\n    </tr>\n    <tr>\n      <th>part</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>LMA2718T421-OA5</th>\n      <td>200.0</td>\n      <td>39</td>\n    </tr>\n    <tr>\n      <th>S15OT421-005</th>\n      <td>150.0</td>\n      <td>35</td>\n    </tr>\n    <tr>\n      <th>LMA2718T381-OC1</th>\n      <td>200.0</td>\n      <td>32</td>\n    </tr>\n    <tr>\n      <th>LMA2718T381-OY1</th>\n      <td>200.0</td>\n      <td>30</td>\n    </tr>\n  </tbody>\n</table>'

In [13]:

def db_to_volt(dbv, ref=1.0):
    v = ref * 10 ** (dbv/20.0)
    return v

def db_from_volt(v, ref=1.0):
    db = 20 * numpy.log10(v/ref)
    return db

def compute_gain(input_spl):
    mic_sensitivity = -38
    headroom = 20.0
    
    # Battery down to 3.0v. Gives 1.5v centerpoint, and a 1.5v peak
    adc_volt = 1.5
    adc_db = db_from_volt(adc_volt) 
    
    mic_level = (mic_sensitivity-(94-input_spl))
    gain = adc_db - (mic_level + headroom)

    return gain

input_levels = pandas.DataFrame({
    'level': [
    50.0,
    60.0,
    70.0,
    80.0,
    90.0,
]})

gains = input_levels.copy()
gains['preamp_gain'] = input_levels.apply(compute_gain).round(2)
gains.set_index('level')

,preamp_gain
level,
50.0,65.52
60.0,55.52
70.0,45.52
80.0,35.52
90.0,25.52


In [15]:
db_to_volt(25)

17.78279410038923

In [21]:

def load_opamps():
    path = '../references/microphones/opamp_specifications.csv'
    df = pandas.read_csv(path, header=1)
    df = df.set_index(['part', 'manufacturer'])
    return df

opamps = load_opamps()
opamps


,,cost,current,noise_density,slewrate,gbp
part,manufacturer,,,,,
LMV321IDBVR,UMW,0.0290,0.06,27.0,0.52,1.00
TLV333,TI,0.2085,0.06,55.0,0.16,0.35
LM321LVIDBVR,TI,0.0320,0.09,40.0,1.50,1.00
GS8621,Gainsil,0.0702,0.25,18.0,1.66,3.00
GS8721,Gailsil,0.0847,1.50,12.0,9.00,11.00
LMV721,Tokmas,0.0732,1.50,11.5,9.00,11.00


In [75]:
db_from_volt(400)

52.04119982655925

In [16]:
db_to_volt(40)

100.0

In [29]:


def opamp_noise_density_max(mic_noise, margin_db=6, bandwidth=20000):
    """
    Returns the maxinum noise density [nV/√Hz]
    """
    preamp_noise_db = mic_noise - margin_db
    m = numpy.sqrt(bandwidth)
    density_v = db_to_volt(preamp_noise_db) / m
    density_nv = density_v / 1e-9

    return density_nv

def test_opamp_noise_density():
    
    d = opamp_noise_density_max(-103, margin_db=10)
    assert int(d) == 15, d

test_opamp_noise_density()

20.0


In [27]:

mics.noisefloor.apply(opamp_noise_density_max)


part
LMA2718T421-OA5    50.059326
LMA2718T381-OC1    35.439289
LMA2718T381-OY1    28.150428
S15OT421-005       31.585300
Name: noisefloor, dtype: float64

In [37]:

def slew_rate(vp, fmax=20000):
    sr_v_s = 2 * numpy.pi * fmax * vp
    sr_v_us = sr_v_s / 1e6
    return sr_v_us

slew_rate(2.0)



0.25132741228718347

In [19]:
def gain_bandwidth(gain, bandwidth=20000, factor=2.0):
    gbp = gain * (bandwidth * factor)
    gbp_mhz = gbp / 1e6
    return gbp_mhz

gain_bandwidth(100, bandwidth=16000)

3.2